In [3]:
# Database connect
import psycopg2

# DataFrames
import pandas as pd
pd.set_option('display.max_columns', 500)

# Plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Cross validation
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.model_selection import KFold, ShuffleSplit


# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

In [4]:
db_info = {
    'user':'MHollowed',
    'database':'real_estate_data'
}
conn = psycopg2.connect(**db_info)
cur = conn.cursor()

In [5]:
def execute_query(q):
    conn.rollback()
    cur.execute(q)
    conn.commit()
    return cur.fetchall(), cur.description

In [115]:
query = '''
    SELECT *
    FROM sales_info s
    LEFT JOIN property_info p
    ON s.major = p.major AND s.minor = p.minor
    ;
'''

query2 = '''
    SELECT *
    FROM sales_info s
    ;
'''


In [116]:
query_result, description = execute_query(query)
column_names = [d.name for d in description]

In [117]:
df = pd.DataFrame(query_result, columns=column_names)

In [118]:
# Change 'DocumentDate' to DateTime
df['document_date'] = pd.to_datetime(df['document_date'])
# Get SaleYear and SaleMonth columns out of the DateTime object
df['sale_year'] = pd.DatetimeIndex(df['document_date']).year
df['sale_year'] -= df['sale_year'].min()
df['sale_month'] = pd.DatetimeIndex(df['document_date']).month

In [119]:
def filter_df(keep_values, col_name, df):
    mask = df[col_name].isin(keep_values)
    return df[mask]

In [120]:
'''
PrincipalUse value of 6 represents Residential buildings
Keep only sales with PrincipalUse value of 6
'''
principal_use_keep_values = [6]
df = filter_df(keep_values=principal_use_keep_values, col_name='principal_use', df=df)

'''
PropertyType value of 1 represents Land Only
PropertyType value of 2 represents Land with New Building
PropertyType value of 3 represents Land with Previously Used Building
Keep only sales with PropertyType value in [1, 2, 3]
'''
property_type_keep_values = [1, 2, 3]
df = filter_df(property_type_keep_values, 'property_type', df)

'''
SaleInstrument value of 3 represents a Statutory Warranty Deed
By using this deed, the seller promises the buyer 
1. The seller is the owner of the property and has the right to sell it
2. No one else is possessing the property
3. There are no encumbrances against the property
4. No one with a better claim to the property will interfere with the transferee’s rights
5. The seller will defend certain claims regarding title to the property
'''
sale_instrument_keep_values = [3]
df = filter_df(sale_instrument_keep_values, 'sale_instrument', df)

In [121]:
# Shuffle dataframe
df = shuffle(df)
df.fillna(0, inplace=True)

In [75]:
for i in df.columns:
    print(i, end=', ')

index, excise_tax_nbr, major, minor, document_date, sale_price, recording_nbr, volume, page, plat_nbr, plat_type, plat_lot, plat_block, seller_name, buyer_name, property_type, principal_use, sale_instrument, af_forest_land, af_current_use_land, af_non_profit_use, af_historic_property, sale_reason, property_class, sale_warning, index, major, minor, bldg_nbr, nbr_living_units, address, building_number, fraction, direction_prefix, street_name, street_type, direction_suffix, zip_code, stories, bldg_grade, bldg_grade_var, sq_ft_1st_floor, sq_ft_half_floor, sq_ft_2nd_floor, sq_ft_upper_floor, sq_ft_unfin_full, sq_ft_unfin_half, sq_ft_tot_living, sq_ft_tot_basement, sq_ft_fin_basement, fin_basement_grade, sq_ft_garage_basement, sq_ft_garage_attached, daylight_basement, sq_ft_open_porch, sq_ft_enclosed_porch, sq_ft_deck, heat_system, heat_source, brick_stone, view_utilization, bedrooms, bath_half_count, bath_3qtr_count, bath_full_count, fp_single_story, fp_multi_story, fp_freestanding, fp_addi

In [122]:
# Setting up features for model:

features = [
    'sale_price',
    'property_type',
    'sale_reason',
    'property_class',
    'sale_year',
    'sale_month',
    'nbr_living_units', 
    'stories',
    'bldg_grade',
    'bldg_grade_var', 
    'sq_ft_1st_floor',
    'sq_ft_half_floor',
    'sq_ft_2nd_floor',
    'sq_ft_upper_floor',
    'sq_ft_unfin_full',
    'sq_ft_unfin_half',
    'sq_ft_tot_living',
    'sq_ft_tot_basement',
    'sq_ft_fin_basement',
    'fin_basement_grade',
    'sq_ft_garage_basement',
    'sq_ft_garage_attached'
]



# # Features:
# sales_feature = [
#     'property_type',
#     'sale_reason',
#     'property_class',
#     'sale_year',
#     'sale_month'
# ]

# property_features = [
#     'nbr_living_units', 
#     'stories',
#     'bldg_grade',
#     'bldg_grade_var', 
#     'sq_ft_1st_floor',
#     'sq_ft_half_floor',
#     'sq_ft_2nd_floor',
#     'sq_ft_upper_floor',
#     'sq_ft_unfin_full',
#     'sq_ft_unfin_half',
#     'sq_ft_tot_living',
#     'sq_ft_tot_basement',
#     'sq_ft_fin_basement',
#     'fin_basement_grade',
#     'sq_ft_garage_basement',
#     'sq_ft_garage_attached'
# ]

# combined_features = sales_feature + property_features

# features_as_is = [
#     'sale_year',
#     'sale_month'
# ]

# features_to_scale = [
#     'sq_ft_1st_floor',
#     'sq_ft_half_floor',
#     'sq_ft_2nd_floor',
#     'sq_ft_upper_floor',
#     'sq_ft_unfin_full',
#     'sq_ft_unfin_half',
#     'sq_ft_tot_living',
#     'sq_ft_tot_basement',
#     'sq_ft_fin_basement',
#     'sq_ft_garage_basement',
#     'sq_ft_garage_attached'
# ]

# dummy_features = [
#     'property_type',
#     'sale_reason',
#     'property_class',
#     'nbr_living_units', 
#     'stories',
#     'bldg_grade',
#     'bldg_grade_var'
# ]





# # Standardize the dataframe
# scalar = StandardScaler().fit(df[features_to_scale])
# df[features_to_scale] = scalar.transform(df[features_to_scale])

# final_df = df[features_as_is + features_to_scale + ['sale_price'] + sales_feature]

# # Get dummy cols
# dummies = pd.get_dummies(df[dummy_features].applymap(str))

# final_df = final_df.append(dummies)


scalar = StandardScaler()


final_df = df[features]
for feat in features_to_scale:
    final_df[feat] = scalar.fit_transform(final_df[feat].values.reshape(-1,1))

    

for feat in dummy_features:
    dummies = pd.get_dummies(df[feat])
    final_df = final_df.append(dummies)
    del final_df[feat]

    
final_df.fillna(0, inplace=True)
# Target value: SalePrice
y = final_df.pop('sale_price')

X = final_df



/Users/MHollowed/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/MHollowed/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/MHollowed/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '>' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/MHollowed/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instanc

In [ ]:
final_df.shape

(5251568, 42)

In [102]:
final_df.shape

(274368, 38)

In [97]:
final_df.shape

(34296, 21)

In [ ]:
# Train Test Split Score

for _ in range(1):
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Random Forest
    model_RF = RandomForestRegressor(n_jobs=-1, n_estimators=100)
    model_RF.fit(X_train, y_train)
    print("Random Forest Score: {}".format(model_RF.score(X_test, y_test)))

# Gradient Booster
# model_GB = GradientBoostingRegressor()
# model_GB.fit(X_train, y_train)
# print("Gradient Booster Score: {}".format(model_GB.score(X_test, y_test)))

In [ ]:
#### Cross Val Score
model_cv_RF = RandomForestRegressor(n_jobs=-1, n_estimators=100)
#model_cv_GB = GradientBoostingRegressor()
print(cross_val_score(model_cv_RF, X, y, cv=3).mean())
#print(cross_val_score(model_cv_GB, X, y, cv=10).mean())

In [343]:
final_df

,sale_year,sale_month,sq_ft_1st_floor,sq_ft_half_floor,sq_ft_2nd_floor,sq_ft_upper_floor,sq_ft_unfin_full,sq_ft_unfin_half,sq_ft_tot_living,sq_ft_tot_basement,sq_ft_fin_basement,sq_ft_garage_basement,sq_ft_garage_attached
6672,2005,1,0.353590,-0.265689,-0.786705,-0.126287,-0.02494,-0.033086,0.568852,1.752471,2.231370,0.606006,-0.940831
10893,2005,7,-2.094142,-0.265689,-0.786705,-0.126287,-0.02494,-0.033086,-1.904539,-0.738421,-0.597294,-0.414967,-0.940831
18359,2001,11,0.129502,-0.265689,-0.786705,-0.126287,-0.02494,-0.033086,-0.705038,-0.738421,-0.597294,-0.414967,0.930997
30062,1984,3,-2.094142,-0.265689,-0.786705,-0.126287,-0.02494,-0.033086,-1.904539,-0.738421,-0.597294,-0.414967,-0.940831
19415,1999,11,0.508728,-0.265689,-0.786705,-0.126287,-0.02494,-0.033086,-0.500471,-0.071844,-0.597294,-0.414967,-0.940831
496,2012,6,-0.422100,-0.265689,-0.786705,-0.126287,-0.02494,-0.033086,-0.202921,0.770147,1.364522,-0.414967,-0.328233
15024,2008,2,0.474253,-0.265689,1.815857,-0.126287,-0.02494,-0.033086,1.080267,-0.738421,-0.597294,-0.414967,1.475529
23693,2004,12,-2.094142,-0.265689,-0.786705,-0.126287,-0.02494,-0.033086,-1.904539,-0.738421,-0.597294,-0.414967,-0.940831
2495,2005,2,-0.129061,-0.265689,1.316529,-0.126287,-0.02494,-0.033086,0.447972,-0.738421,-0.597294,-0.414967,0.420499
27596,2015,7,-0.473812,-0.265689,0.862593,-0.126287,-0.02494,-0.033086,-0.016951,-0.738421,-0.597294,-0.414967,0.182266


In [110]:
df.columns

Index(['index', 'excise_tax_nbr', 'major', 'minor', 'document_date',
       'sale_price', 'recording_nbr', 'volume', 'page', 'plat_nbr',
       'plat_type', 'plat_lot', 'plat_block', 'seller_name', 'buyer_name',
       'property_type', 'principal_use', 'sale_instrument', 'af_forest_land',
       'af_current_use_land', 'af_non_profit_use', 'af_historic_property',
       'sale_reason', 'property_class', 'sale_warning', 'index', 'major',
       'minor', 'bldg_nbr', 'nbr_living_units', 'address', 'building_number',
       'fraction', 'direction_prefix', 'street_name', 'street_type',
       'direction_suffix', 'zip_code', 'stories', 'bldg_grade',
       'bldg_grade_var', 'sq_ft_1st_floor', 'sq_ft_half_floor',
       'sq_ft_2nd_floor', 'sq_ft_upper_floor', 'sq_ft_unfin_full',
       'sq_ft_unfin_half', 'sq_ft_tot_living', 'sq_ft_tot_basement',
       'sq_ft_fin_basement', 'fin_basement_grade', 'sq_ft_garage_basement',
       'sq_ft_garage_attached', 'daylight_basement', 'sq_ft_open_porch',
 

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3506 entries, 2 to 9999
Data columns (total 78 columns):
index                    3506 non-null int64
excise_tax_nbr           3506 non-null int64
major                    3506 non-null int64
minor                    3506 non-null int64
document_date            3506 non-null datetime64[ns]
sale_price               3506 non-null int64
recording_nbr            3506 non-null object
volume                   3506 non-null object
page                     3506 non-null object
plat_nbr                 3506 non-null object
plat_type                3506 non-null object
plat_lot                 3506 non-null object
plat_block               3506 non-null object
seller_name              3506 non-null object
buyer_name               3506 non-null object
property_type            3506 non-null int64
principal_use            3506 non-null int64
sale_instrument          3506 non-null int64
af_forest_land           3506 non-null object
af_current_use_land